In [ ]:
import pandas as pd
import os
import re
import numpy as np
from numpy import log 

#librerias para texto
import nltk #Natural Language Tool Kit

#importamos tokenizador de texto
from nltk.tokenize import word_tokenize 

#stemmer mediate algoritmo de porter
from nltk.stem import PorterStemmer 

#stemmer de libreria snowball
from nltk import SnowballStemmer 

#para conteo de vectores de texto
from sklearn.feature_extraction.text import CountVectorizer

#para reemplazo de caracteres especiales latinos
import unidecode 
import unicodedata

#libreria gensim para complementar trabajo con texto
import gensim

#modulo LDA de sklearn
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings("ignore")


In [ ]:
#Cambiamos directorio
dir=os.chdir('C:/Users/rfern/Desktop/Modulo 9/Codigos y datos')

In [ ]:
#importamos modulo para lectura del pdf
from PyPDF2 import PdfFileReader
file = open('new-testament-83291-spa.pdf', 'rb')
#cargamos pdf en un objeto PyPDF2.pdf.PdfFileReader
reader = PdfFileReader(file)
print(reader)

In [ ]:
#creamos string vacío para almacenar resultados
resultado = '' 

#generamos ciclo para cada numero de pagina
for i in range(reader.numPages ):
    
    #creamos escalar que indexa el numero de pagina
    pagina = reader.getPage(i)
    
    #sumamos el texto extraido para la pagina indexada al resultado
    resultado += ' '+ pagina.extractText() + ' '

In [ ]:
regex=r'(?<=\d\s).*?(?=\.)'

In [ ]:
#implementamos la busqueda mediante metodo .findall
versiculos=re.findall(regex, resultado, re.DOTALL)

#imprimimos el numero de articulos de la busqueda
print(len(versiculos), '\n')

In [ ]:
#imprimimos cada uno de los articulos de la busqueda
for vers in range(len(versiculos)):
    print(vers+1, versiculos[vers], '\n')

In [ ]:
#Pasamos la lista de articulos a un dataframe
df=pd.DataFrame(versiculos, columns=['contenido_crudo'])
df.info()

In [ ]:
df.head(50)

In [ ]:
#importamos stopwords
from nltk.corpus import stopwords 

#descargamos stopwords
nltk.download('stopwords')

#creamos lista de stopwords en castellano
stopwords=stopwords.words('spanish')

#normalizamos stopwords, removemos tildes y otros caracteres latinos para el match con texto normalizado
#creamos lista vacía que almacenara palabras normalizadas
stop_words=[]
#ciclo en el que cada palabra
for word in stopwords:
    #es normalizada
    word_norm = unicodedata.normalize('NFD', word).encode('ascii', 'ignore').decode("utf-8")
    
    #y almacenada en la nueva lista
    stop_words.append(word_norm)

#Con la lematización pasa algo similar, cortar palabras puede hacernos perder su riqueza.
snowball = SnowballStemmer(language='spanish')
#instanciamos el lematizador de porter
#porter= PorterStemmer(languaje='spanish')

#creamos funcion que limpia y normaliza texto de la descripcion web
def limpieza(texto):
    #pasamos a minusculas
    texto_norm=texto.lower()
    
    #removemos espacios al final y al inicio de cada cadena
    texto_norm=texto_norm.strip()
    
    #removemos dobles espacios
    texto_norm=re.sub('\s',' ', texto_norm)
    
    #nuevo_texto = unidecode.unidecode(nuevo_texto)
    texto_norm = unicodedata.normalize('NFD', texto_norm).encode('ascii', 'ignore').decode("utf-8")
    
    #retenemos caracteres alfanumericos: ojo que incluimos la ñ
    texto_norm=re.sub('[^A-Za-z]+', ' ', texto_norm)
    
    #tokenizamos texto: convertir en una lista una cadena
    texto_norm = word_tokenize(texto_norm)
    
   
    #removemos stopwords
    texto_norm=[word for word in texto_norm if not word in stop_words]
    
    #lematizamos texto con el stemmer de snowbal
    #texto_norm=[snowball.stem(word) for word in texto_norm]
    
    # Eliminación de terminos de largo 1
    texto_norm = [token for token in texto_norm if len(token) > 3] 
   
    #resultado de la funcion
    return(texto_norm)

In [ ]:
#implementamos la funcion de limpieza y normalizacion
df['contenido_limpio'] = df['contenido_crudo'].apply(lambda x: limpieza(x))

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detok=TreebankWordDetokenizer()
df['contenido_limpio_detok']=df['contenido_limpio'].map(lambda x: detok.detokenize(x))

In [ ]:
df.head()

In [ ]:
df['largo'] = df['contenido_limpio'].map(lambda x: len(x))

In [ ]:
df.head()

In [ ]:
df.largo.describe(percentiles=[.1,.2,.3,.5,.6,.7,.8,.9])

In [ ]:
df=df[df['largo']>=4]

In [ ]:
#otra manera de preprocesar el texto
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        
data = df.contenido_limpio_detok.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

In [ ]:
#diseñamos funcion para la generacion de ngramas
def generate_N_grams(text,ngram):
    temp=zip(*[text[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans


In [ ]:
#creamos lista de trigramas
data_ngram=[]
for i in data_words:
    t=generate_N_grams(i, 2)
    data_ngram.append(t)

In [ ]:
data_ngram

In [ ]:
#importamos modulo gensim para la generacion del corpus
import gensim.corpora as corpora
# creamos diccionario
id2word = corpora.Dictionary(data_ngram)

# creamos corpus
texts = data_ngram

# creamos DTM
corpus = [id2word.doc2bow(text) for text in texts]

# Vista del corpus para el primer elemento
print(corpus[:1])

In [ ]:
#Contabilizamos el numero de nucleos de la CPU
import multiprocessing
workers=multiprocessing.cpu_count()
workers

In [ ]:
#implementamos modelo de latent diriechlect allocation
lda_model = gensim.models.LdaMulticore(corpus=corpus,     #ingresamos corpus
                                       workers=workers-1 ,#numero de nucleos para procesamiento paralelo
                                       id2word=id2word,   #ingresamos la indexacion de ngramas
                                       num_topics=8,      #numero de topidos
                                       random_state=100,  #semilla aleatoria para replicabilidad de resultados
                                       chunksize=10,      #numero de documentos que serán utilizados en cada fase de entrenamient
                                       alpha=0.5,         #parametro alfa
                                       eta=0.3)           #    

In [ ]:
#calculamos el indice de coherencia
from gensim.models import CoherenceModel
coherencia_model_lda = CoherenceModel(model=lda_model, 
                                      texts=data_ngram, 
                                      dictionary=id2word, 
                                      coherence='c_v')

#obtenemos el indice de coherencia
coherencia = coherencia_model_lda.get_coherence()

#imprimimos resultado
print(coherencia)

In [ ]:
#importamos librerias y modulos para la visualizacion
import pyLDAvis
import pyLDAvis.gensim_models

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
# visualizamos topicos
vis= pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'modelo.html')

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# Funcion para implementacion de modelo

def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                          workers=workers-1,
                                          id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ngram, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Set de validacion'] = {}
# rango de topicos
min_topics = 2
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# rango de valores para parametro alfa
alpha = list(np.arange(0.1, 1, 0.1)) 
alpha.append('symmetric')
alpha.append('asymmetric')
# rango de valores para parametro beta
beta = list(np.arange(0.2, 1, 0.1))
beta.append('symmetric')
## set de validacion
##numero de documentos
num_of_docs = len(corpus)
#
##utilizamos el 75% del corpus
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), corpus]
#
##titulos
corpus_title = ['75% Corpus', '100% Corpus']
#
##diccionario de resultados
model_results = {'Set de validacion': [],
                'Topicos': [],
                 'Alfa': [],
                 'Beta': [],
                 'Coherencia': []
                }
## iteraciones sobre los parametros
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iteramos sobre corpus de validacion
    for i in range(len(corpus_sets)):
        # iteramos sobre el numero de topicos
        for k in topics_range:
            # sobre parametro alfa
            for a in alpha:
                # sobre parametro beta
                for b in beta:
                    # calculamos el coeficiente de coherencia
                    cv = compute_coherence_values(corpus=corpus_sets[i], 
                                                  dictionary=id2word, 
                                                  k=k, 
                                                  a=a, 
                                                  b=b)
                    # guardamos resultados
                    model_results['Set de validacion'].append(corpus_title[i])
                    model_results['Topicos'].append(k)
                    model_results['Alfa'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherencia'].append(cv)
                    
                    pbar.update(1)
    resultados=pd.DataFrame(model_results)
    resultados.to_csv('resultados_lda.csv', index=False)
    pbar.close()